In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [0]:
train = pd.read_csv("/content/drive/My Drive/Machine _Hack/janta-hack-CV/train/train.csv")

In [5]:
train.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [6]:
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img("/content/drive/My Drive/Machine _Hack/janta-hack-CV/train/images/"+train['image_names'][i], grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 1646/1646 [12:27<00:00,  2.20it/s]


In [0]:
y=train['emergency_or_not'].values
y = to_categorical(y)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state = 294)

# VGG 16 Model

In [9]:
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.applications import VGG16
from keras.models import Model

image_size = 224
input_shape = (image_size, image_size, 3)

epochs = 5
batch_size = 16

pre_trained_model = VGG16(input_shape=input_shape, include_top=False, weights="imagenet")
    
for layer in pre_trained_model.layers[:15]:
    layer.trainable = False

for layer in pre_trained_model.layers[15:]:
    layer.trainable = True
    
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
    
# Flatten the output layer to 1 dimension
x = GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(2, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

58892288/58889256 [==============================] - 6s 0us/step
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

# Resnet 101 Model

In [0]:
from keras.applications import ResNet101

In [0]:
image_size = 224
input_shape = (image_size, image_size, 3)
epochs = 5
batch_size = 16
# pre_trained_model = ResNet101(input_shape=input_shape, include_top=False, weights="imagenet")
my_new_model = Sequential()
my_new_model.add(ResNet101(input_shape=input_shape, include_top=False, weights="imagenet"))
my_new_model.add(Dense(1024, activation='relu'))
my_new_model.add(Dense(512, activation='relu'))
my_new_model.add(Dropout(0.5))
my_new_model.add(Dense(256, activation='relu'))
my_new_model.add(Dense(2, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

In [16]:
my_new_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

my_new_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101 (Model)            (None, 7, 7, 2048)        42658176  
_________________________________________________________________
dense_6 (Dense)              (None, 7, 7, 1024)        2098176   
_________________________________________________________________
dense_7 (Dense)              (None, 7, 7, 512)         524800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 7, 7, 512)         0         
_________________________________________________________________
dense_8 (Dense)              (None, 7, 7, 256)         131328    
_________________________________________________________________
dense_9 (Dense)              (None, 7, 7, 2)           514       
Total params: 45,412,994
Trainable params: 2,754,818
Non-trainable params: 42,658,176
__________________________________

# Model Training

In [0]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [26]:
model.fit(X_train, y_train, epochs=50,  validation_data=(X_test, y_test),callbacks=[es,mc])

Train on 1399 samples, validate on 247 samples
Epoch 1/50
1399/1399 [==============================] - 17s 12ms/step - loss: 0.1558 - accuracy: 0.9467 - val_loss: 0.2418 - val_accuracy: 0.9008

Epoch 00001: val_accuracy improved from 0.89879 to 0.90081, saving model to best_model.h5
Epoch 2/50
1399/1399 [==============================] - 17s 12ms/step - loss: 0.1475 - accuracy: 0.9417 - val_loss: 0.2352 - val_accuracy: 0.8968

Epoch 00002: val_accuracy did not improve from 0.90081
Epoch 3/50
1399/1399 [==============================] - 17s 12ms/step - loss: 0.1401 - accuracy: 0.9485 - val_loss: 0.2336 - val_accuracy: 0.8968

Epoch 00003: val_accuracy did not improve from 0.90081
Epoch 4/50
1399/1399 [==============================] - 17s 12ms/step - loss: 0.1357 - accuracy: 0.9489 - val_loss: 0.2322 - val_accuracy: 0.8988

Epoch 00004: val_accuracy did not improve from 0.90081
Epoch 5/50
1399/1399 [==============================] - 17s 12ms/step - loss: 0.1284 - accuracy: 0.9560 - val_

## Generic Model

In [0]:
# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_uniform',input_shape=(28,28,1)))
# model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer='he_uniform'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.2))
# model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(2, activation='softmax'))

In [0]:
# model.compile(loss='binary_crossentropy',optimizer="sgd",metrics=['accuracy'])

In [0]:
# model.fit(X_train, y_train, epochs=500,  validation_data=(X_test, y_test))

# Graphs

In [0]:
def plot_model_history(model_history, acc='acc', val_acc='val_acc'):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    axs[0].plot(range(1,len(model_history.history[acc])+1),model_history.history[acc])
    axs[0].plot(range(1,len(model_history.history[val_acc])+1),model_history.history[val_acc])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history[acc])+1),len(model_history.history[acc])/10)
    axs[0].legend(['train', 'val'], loc='best')
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

# Inference of the model

In [0]:
test = pd.read_csv("/content/drive/My Drive/Machine _Hack/janta-hack-CV/test/test.csv")

In [33]:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('/content/drive/My Drive/Machine _Hack/janta-hack-CV/test/images/'+test['image_names'][i])
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

100%|██████████| 706/706 [00:02<00:00, 346.38it/s]


In [0]:
from keras.models import load_model
model = load_model("/content/best_model.h5")

In [0]:
prediction = model.predict(test)

In [0]:
classs = np.argmax(prediction, axis = 1)

In [0]:
sample = pd.read_csv("/content/drive/My Drive/Machine _Hack/janta-hack-CV/sample_submission_yxjOnvz.csv")

In [0]:
sample['emergency_or_not'] = classs
sample.to_csv("Final_sub_resnet"+'.csv', header=True, index=False)
